In [3]:
# Custom style
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
import os
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import random

In [5]:
# Save this to make sure files are backed up
# git config --global credential.helper store
# os.system("git add . && git commit -m 'generic work progress' && git push")

# Queuing-as df first
To Do:  
* run better variables for timing customer incident rates
* more interesting call times
* factor in timed breaks time for reps
* factor in random breaks for reps
* bring in new reps for a timeframe
* let reps leave for the days

Later: 
* Abstract this to language types
  
Later: transaction logs as classes with add node function?

In [6]:
num_customerz = 200
time_periodz = 1000
num_repz = 5

In [10]:
def generate_customer_table(num_customers, time_periods, seed=None):
    if seed is not None:
        np.random.seed(seed)

    df_customers = pd.DataFrame([i for i in range(1,num_customers+1)], columns=['customer_id'])
    df_customers['required_call_time'] = [np.random.choice([1,2,3,4,5]) for i in range(num_customers)]
    df_customers['time_in']=[np.random.choice(range(1,time_periods)) for i in range(num_customers)]
    df_customers['time_out']=[0 for i in range(num_customers)]
    df_customers['time_assigned'] = [0 for i in range(num_customers)]
    df_customers['assigned_rep'] = ['none' for i in range(num_customers)]

    df_customers=df_customers.sort_values(by='time_in')
    df_customers['customer_id'] = [i for i in range(1,num_customers+1)]
    
    return df_customers

def generate_qs(time_periods):

    df_q = pd.DataFrame([0 for x in range(time_periods)], columns=['customer_q_list'])
    df_q['customer_q_list'] = [[] for x in range(time_periods)]
    df_q['update_event'] = [0 for x in range(time_periods)]

    return df_q

def generate_reps(num_reps, time_periods):
    reps_list = ['rep' + str(i) + '_busy' for i in range(1,num_reps+1)]
    df_reps = pd.DataFrame(columns=reps_list, index=range(time_periods))
    for rep in reps_list:
        df_reps[rep] = 0

    return df_reps

In [11]:
def run_day(df_customers, df_q, df_reps, iteration, time_periods):
    _customer_q = []
    _cust_in_time = list(df_customers.time_in.values)
    _reps_avail = []
    _reps_busy = []
    
    for t in range(time_periods): # Step through each time period 
        if t in _cust_in_time: # If one or more custommers enter at that time
            _customer_q += list(df_customers[df_customers.time_in==t].customer_id.values) # Append them to the queue

        for customer_qued in _customer_q: # look through the queue

            for rep in df_reps.columns:
                if df_reps.at[t, rep] ==0:
                    _req_call_time = int(df_customers[df_customers.customer_id==customer_qued]['required_call_time'])
                    df_customers.at[df_customers.customer_id==customer_qued, ['time_assigned', 'time_out',  'assigned_rep']] = [t, t+_req_call_time , rep]

                    df_reps.loc[t: t + _req_call_time, rep] = 1
                    break
        df_q.at[t,'customer_q_list'] = _customer_q
        
    df_customers['wait_time'] = df_customers.time_assigned - df_customers.time_in
    df_customers['iteration'] = iteration

    df_txn_log = pd.concat([df_q, df_reps],axis=1)
    df_txn_log['num_in_q'] = df_txn_log.customer_q_list.apply(lambda x: len(x))
    df_txn_log['iteration'] = iteration
    df_txn_log = df_txn_log.iloc[:df_customers.time_out.max()+1, :]
    return df_customers, df_txn_log

In [12]:
%%time
df_customers, df_txn_log = run_day(generate_customer_table(num_customers=num_customerz, time_periods=time_periodz, seed=42), 
                                   generate_qs(time_periodz), 
                                   generate_reps(num_repz, time_periodz), 
                                   iteration=1, 
                                  time_periods=time_periodz)

"""
Timed: 5 min 30 s. 2min 44 with basic list cut
num_customerz = 10*24
time_periodz = 24*60
num_repz = 50
"""

CPU times: user 2min 44s, sys: 141 ms, total: 2min 44s
Wall time: 2min 44s


In [13]:
df_txn_log.tail()

customer_q_list  update_event  \
1435  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...             0   
1436  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...             0   
1437  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...             0   
1438  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...             0   
1439  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...             0   

      rep1_busy  rep2_busy  rep3_busy  rep4_busy  rep5_busy  rep6_busy  \
1435          1          1          1          1          1          1   
1436          1          1          1          1          1          1   
1437          1          1          1          1          1          1   
1438          1          1          1          1          1          1   
1439          1          1          1          1          1          1   

      rep7_busy  rep8_busy    ...      rep43_busy  rep44_busy  rep45_busy  \
1435          1          1    ...               1           1           1   
1436          1          1    ...               1           1           1   
1437          1          1    ...               1           1           1   
1438          1          1    ...               1           1           1   
1439          1          1    ...               1           1           1   

      rep46_busy  rep47_busy  rep48_busy  rep49_busy  rep50_busy  num_in_q  \
1435           1           1           1           1           1       240   
1436           1           1           1           1           1       240   
1437           1           1           1           1           1       240   
1438           1           1           1           1           1       240   
1439           1           1           1           1           1       240   

      iteration  
1435          1  
1436          1  
1437          1  
1438          1  
1439          1  

[5 rows x 54 columns]

In [14]:
df_customer.head()

NameError: name 'df_customer' is not defined

# Run a whole week

In [10]:
for i in range(2,10):
    df_a, df_b = run_day(generate_customer_table(), 
                                   generate_qs(), 
                                   generate_reps(), 
                                   iteration=i)
    df_customers = pd.concat([df_customers, df_a])
    df_txn_log = pd.concat([df_txn_log, df_b])

NameError: name 'time_periods' is not defined

In [32]:
# Wait Times
np.quantile(df_customers.wait_time, [.25, .5, .99])

array([ 8.  , 18.  , 41.01])

In [33]:
# Rep utilization rates
df_txn_log.loc[:, ['rep1_busy', 'rep2_busy', 'rep3_busy']].mean()


rep1_busy    0.977866
rep2_busy    0.969170
rep3_busy    0.938340
dtype: float64